1. [Santander Fast Compact Solution](https://www.kaggle.com/gpreda/santander-fast-compact-solution)
Go to Kaggle website, resgister, attend competition, and make groups.

In [ ]:
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Nadam

warnings.filterwarnings('ignore')

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import gc 
gc.collect()

In [ ]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

In [ ]:
mean_arr_test=[test_df.iloc[i,1:].mean() for i in range(len(test_df))]
mean_std_test=[test_df.iloc[i,1:].std() for i in range(len(test_df))]
large_10_test=[np.sort(test_df.iloc[i,1:].values)[-10:].mean() for i in range(len(test_df))]
small_10_test=[np.sort(test_df.iloc[i,1:].values)[:10].mean() for i in range(len(test_df))]

In [ ]:
test_df['var_200']=mean_arr_test
test_df['var_201']=mean_std_test
test_df['var_202']=large_10_test

test_df['var_203']=small_10_test

In [ ]:
mean_arr_train=[train_df.iloc[i,2:].mean() for i in range(len(train_df))]
mean_std_train=[train_df.iloc[i,2:].std() for i in range(len(train_df))]
large_10_train=[np.sort(train_df.iloc[i,2:].values)[-10:].mean() for i in range(len(train_df))]
small_10_train=[np.sort(train_df.iloc[i,2:].values)[:10].mean() for i in range(len(train_df))]

In [ ]:
train_df['var_200']=mean_arr_train
train_df['var_201']=mean_std_train
train_df['var_202']=large_10_train

train_df['var_203']=small_10_train

In [ ]:
gc.collect()

Features
---
1. target, predicted feature
2. var_n, b=0, ...,199, **200** given features
3. new features:
```
      var_200: mean of 200 features
      var_201: std  of 200 eatures
      var_202: mean of largest 10 features
      var_203: mean of smallest 10 features
```  
4. `df.iloc[i,j:]`: the i-th row, and all the columns starting from j-th column (feature)

In [ ]:
target_feature=['target']
var_features=train_df.columns.tolist()[-204:]

In [ ]:
train_df[['var_200','var_201','var_202','var_203']].hist()

In [ ]:

xgb_params = {'eta': 0.01, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.8, 
              'gamma':  5,'lambda':1,'min_child_weight' : 50,  
               'colsample_bylevel' : 0.7,'alpha':0,'booster' : "gbtree",
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}

features = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

In [ ]:
num_round = 1000000
REPEATED=4

folds = StratifiedKFold(n_splits=REPEATED, shuffle=False, random_state=2019)
oof = np.zeros(len(train_df))
predictions_xgb = np.zeros(len(test_df))

In [ ]:
# 204-features category
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = xgb.DMatrix(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = xgb.DMatrix(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    watchlist = [(trn_data, 'train'), (val_data, 'valid')]
    num_round = 5000
    clf_xgb = xgb.train(xgb_params, trn_data, num_round,watchlist,  early_stopping_rounds = 50 ,verbose_eval=100)
    oof[val_idx] = clf_xgb.predict(xgb.DMatrix(train_df.iloc[val_idx][features]), ntree_limit=clf_xgb.best_ntree_limit+50)
    predictions += clf_xgb.predict(xgb.DMatrix(test_df[features]), ntree_limit=xgb_model.best_ntree_limit+50) / folds.n_splits
print("CV score: {:<8.5f}".format(roc_auc_score(target, oof_xgb)))


In [ ]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = predictions
sub.to_csv("output/submission-l1-l6-1_204_features.csv", index=False)

In [ ]:
import plotly.plotly as py
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks
# offline mode
cufflinks.go_offline()
# Set the global theme for cufflinks
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

In [ ]:
# 11-16-1
sub.hist()

In [ ]:
# 11-15-1
sub.hist()